In [1]:
import numpy as np
import pandas as pd
import os, sys
import pickle
import requests, bs4
from datetime import datetime, date, timedelta

In [2]:
os.chdir('/Users/ahelgeso/Documents/GitHub/bball')
os.getcwd()
db = pd.read_csv('2018stats.csv')

In [3]:
url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
res = requests.get(url, 'html_parser')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')
game_soup = soup.find('table',{'id':'schedule'})

NameError: name 'year' is not defined

In [29]:
db['Date'] = db['Date'].apply(str)
db['Date'] = pd.to_datetime(db.Date, format= "%Y%m%d")
db = db.sort_values(by = ['Date']).reset_index(drop = True)

In [79]:
def date_formatter(date):

        if len(str(date.month)) != 2:
            month = '0'+ str(date.month)
        else:
            month = str(date.month)
        
        if len(str(date.day)) != 2:
            day = '0' + str(date.day)
        else:
            day = str(date.day)
        
        newdate = str(date.year) + month + day
        return newdate

In [119]:
a = pd.Timedelta(datetime(2019, 4, 20) - datetime(2019, 3, 30))
s = pd.Series(pd.date_range(datetime(2019, 3, 30), periods = a.days, freq = 'D'))
months = list(set([datetime.strftime(date, '%B').lower() for date in s]))



In [120]:
months

['march', 'april']

In [124]:
offseason = [5, 6, 7, 8, 9]
today = datetime.today()
year = today.year + 1 if today.month >=10 else today.year
year

2019

In [ ]:
def update(db, today = datetime.today()):
    
    '''
    1. Order the current db by date
    2. Check the last date that data was scraped, find time-delta
        - If last scraped date was after current season, exception to make new db
    3. Create a day-by-day timedelta (string?) list 
    4. Scrape raw stats data from bball-reference by date
        - for date in daterange:
            if month of date is in offseason, pass
            scrape nba games schedule for given month
                store game dates in a list
                store team names in a list
                get scores on this pass
            scrape individual game pages from gamedate/home teams 
        - Get team and opponent stats at same time, faster than indexing
    5. Change data types to correct type
        - String date to datetime
        - Points to int
        - four factors to float 
        - ORB, TOV / 100.0
        - Location 1/0
    6. Do math for running averages
    7. Scrape line data from sportsbookreview by date
        - Odds to int, line to float
    '''
    
    if db[db['Game Number'] == 82].sum() == 30:
        return "Season ended; Create new database."
    
    ### 1 ### Sort db by dates
    
    # Formats webscraped dates to match bball reference URLs for later scraping
    def date_formatter(date):

        if len(str(date.month)) != 2:
            month = '0'+ str(date.month)
        else:
            month = str(date.month)
        
        if len(str(date.day)) != 2:
            day = '0' + str(date.day)
        else:
            day = str(date.day)
        
        newdate = str(date.year) + month + day
        return newdate
    
    # Check if dates are already datetime objects
    fixdates = False
    for date in db['Date']:
        if type(date) != pd._libs.tslib.Timestamp:
            fixdates = True
            break
            
    # Turn dates into datetime objects
    if fixdates:
        db['Date'] = db['Date'].apply(str)
        db['Date'] = pd.to_datetime(db.Date, format= "%Y%m%d")
        
    # Sort db every time, i don't see why not
    db = db.sort_values(by = ['Date']).reset_index(drop = True)
    
    # Data is now ordered by date
    
    ### 2 ### Get timedelta of days between last update
    
    # Find difference between date last updated and current date, get range of dates
    last_date = db['Date'].iloc[-1]
    d = pd.Timedelta(today - db['Date'].iloc[-1])
    daterange = pd.Series(pd.date_range(db['Date'].iloc[-1], periods = d.days, freq = 'D'))
    
    # Define offseason months so don't search those; year accounts for updating between seasons
    offseason = ['may', 'june', 'july', 'august', 'september']
    months = list(set([datetime.strftime(date, '%B').lower() for date in daterange if datetime.strftime(date, '%B').lower() not in offseason]))
    years = [today.year + 1 if month in ['october', 'november', 'december'] else today.year for month in months]

    
    ### 3 ### Get all games played in delta months, then compare list to last entry in db
          ### Also compile teams and opponents at this time to make scrape urls easier to build
    
    for month in months:
        if month not in offseason:
            url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html'
            res = requests.get(url, 'html_parser')
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text, 'html.parser')
            game_soup = soup.find('table',{'id':'schedule'})
        
            # Pull regular season game dates, generate list, turn it into datetime objects
            datesoup = game_soup.find_all(attrs = {'data-stat':'date_game'})
            gamedates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]
            game_dates = [datetime.strptime(date, '%a, %b %d, %Y') for date in gamedates]
            game_dates = [date for date in game_d]
            
    points = []
    location = []
    efgs = []
    tovpct = []
    orbpct = []
    ftr = []
    
#    for date in daterange:
        

    # Get just regular season stats
    reg_season = soup.find('table',{'id':'games'})
        
    # Pull regular season game dates, generate list, turn it into datetime objects
    datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
    gamedates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]
    game_dates = [datetime.strptime(date, '%a, %b %d, %Y') for date in gamedates]
        
        
    # Find regular season game locations, translate to 1 = home, 0 = away
    souplocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
    locs = [entry.get_text() for entry in souplocs if entry.get_text() == '' or entry.get_text() == '@']
    game_locs = list(map(lambda x: 1 if x =='' else 0, locs))
    
    # Find opponent team names
    oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
    opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
    opp_abbrev = [team_abbrevs[opp] for opp in opps]
        
    # Get Results
    soupresults = reg_season.find_all(attrs = {'data-stat':'game_result'})
    results = [entry.get_text() for entry in soupresults if entry.get_text() == 'W' or entry.get_text() == 'L']
    rslts = list(map(lambda x: 1 if x =='W' else 0, results))

    # Get game stats
    for i, gamedate in enumerate(game_dates):
            
        date = date_formatter(gamedate)
        dates.append(date)
            
        if game_locs[i] == 1:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[1].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[1].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[1].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[1].get_text())
        else:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team_abbrevs[opps[i]]}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[0].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[0].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[0].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[0].get_text())

        print("Date: " + date + " Opponent: " + team_abbrevs[opps[i]] + " Result: " + str(rslts[i]) + " EFG%: " + efgs[i] + " TOV%: " + tovpct[i] + " ORB%: " + orbpct[i] + " FTR: " + ftr[i])
    
    teamlist= [team] * 82
    data = {'Team': teamlist, "Game Number":list(range(1,83)), "Result":rslts, "Date": dates, "Opponent": opp_abbrev, "EFG%": efgs, "TOV%" : tovpct, "ORB%":orbpct, "FTR":ftr}
    df = pd.DataFrame.from_dict(data)
    df['EFG%'] = df['EFG%'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%'].apply(lambda x: float(x))
    df['ORB%'] = df['ORB%'].apply(lambda x: float(x))
    df['FTR'] = df['FTR'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%']/100.0
    df['ORB%'] = df['ORB%']/100.0

    
    return df
    